In [5]:
import pulp
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable

In [6]:
df = pd.read_csv("../data/shift_dataBase_testing.csv")
df.head()

,staff_id,name,date,morning,afternoon,night
0,1001,Yan Shin Shein,2025-07-07,False,True,True
1,1001,Yan Shin Shein,2025-07-11,False,True,True
2,1001,Yan Shin Shein,2025-07-12,True,True,False
3,1001,Yan Shin Shein,2025-07-16,True,False,True
4,1001,Yan Shin Shein,2025-07-18,False,True,True
